In [3]:
import pandas as pd
from ast import literal_eval

In [4]:
def correctly_read_csv(fname):
    prep_df = pd.read_csv(fname, converters={"tokens_rep": literal_eval, "tokens": literal_eval, "reference": literal_eval})
    return prep_df

In [5]:
prep_df = correctly_read_csv("processed_data.csv")

In [6]:
raw_texts = prep_df['raw_text']
similar_texts = prep_df['reference']

In [7]:
prep_df.head()

,doc_index,raw_text,tokens_rep,tokens_num,reference
0,1,梅雨季來臨文旦黑點病易發生請注意病徵以及早加強防治措施5月已進入梅雨季節近日連續降雨為文旦黑...,"[梅, 雨季, 來臨, 麻豆文旦, 黑點病, 易, 發生, 請, 注意, 病徵, 以, 及早...",150,[]
1,10,天氣多變溫差大近山區及偏施氮肥田區稻熱病發病較為嚴重籲請農友注意防治花蓮區農改場防檢局及田邊...,"[天氣, 多變, 溫差, 大近, 山區, 及, 偏施, 氮肥田區, 稻熱病, 發病, 較為,...",288,[]
2,1000,新聞稿-稻熱病進入好發季節防檢局籲請農友加強防範行政院農業委員會動植物防疫檢疫局以下簡稱防檢...,"[新聞稿, -, 稻熱病, 進入, 好, 發, 季節, 防檢局, 籲請, 農友, 加強, 防...",420,"[1005, 1023]"
3,1005,稻熱病進入好發季節防檢局籲請農友加強防治農委會防檢局表示自3月起全國各地水稻生長陸續進入分蘗...,"[稻熱病, 進入, 好, 發, 季節, 防檢局, 籲請, 農友, 加強, 防治, 農委會, ...",408,"[1000, 1023]"
4,1007,乍暖還寒防檢局籲請農友加強防治稻熱病農委會防檢局表示全國各地水稻生長陸續進入分蘗期因逢暖冬且...,"[乍暖還寒, 防檢局, 籲請, 農友, 加強, 防治, 稻熱病, 農委會, 防檢局, 表示,...",379,"[438, 893, 1000, 1005, 1015, 1023]"


### Create TSV dataset where each row is of format {doc1}\t{doc2}\t{is_similar}
#### 1. Get all similar pairs

In [8]:
id_to_ind = {prep_df['doc_index'][i]:i for i in range(len(prep_df['doc_index']))}

In [9]:
def get_raw_doc_by_id(doc_id):
    ind = id_to_ind[doc_id]
    return raw_texts[ind]

In [10]:
get_raw_doc_by_id(10)

'天氣多變溫差大近山區及偏施氮肥田區稻熱病發病較為嚴重籲請農友注意防治花蓮區農改場防檢局及田邊好幫手關心您清明時節剛過葉稻熱病在花蓮地區開始拉警報花蓮區農業改良場積極監測注意玉里富里等主要水稻生產區觀察到往年稻熱病好發田區如玉里三民大禹等偏施氮肥與近山區發病較為嚴重在稻熱病疫情燈號系統中已亮起黃燈屬於應特別注意防治之區域花蓮農改場籲請農友應多注意田間稻熱病發生情形即時掌握防治時機以免病害大流行在水稻肥培管理上應注意三要素配合若為栽培高雄145台中192等需肥量較少的品種追肥切勿施用過多氮肥以免造成植株徒長葉片柔弱因而增加染病機會如已施用較多氮肥水稻葉色濃綠者則可利用排灌水方式稀釋過多肥份若已發現稻熱病發生則應停施氮肥在使用農藥時應遵守植保手冊推薦用藥相關規定及注意事項有機栽培的農友則可於插秧一個月後開始以亞磷酸1000倍及微生物製劑等資材連續噴施2至3次進行預防以減輕發病建議用藥75%三賽唑可濕性粉劑3,000倍41.7三賽唑水懸劑1,500倍40%亞賜圃乳劑1,000倍加展著劑新利農3000倍5%嘉賜黴素溶液2,000倍45喜樂克拉乳劑1,200倍加展著劑CS-72000倍或50%護粒松乳劑1,000倍等若農民有相關疑問歡迎洽詢花蓮區農業改良場作物環境課植物保護研究室電話03-8521108轉360390或病蟲害防治服務專線03-8535915'

In [11]:
def remove_n_t(string):
    string = string.replace('\n','')
    string = string.replace('\t','')
    string = string.replace('\r','')
    return string

In [12]:
def get_all_similar_pairs(text_col, similar_col):
    all_similar_pairs_list = []
    for i in range(len(text_col)):
        for j in range(len(similar_col[i])):
            all_similar_pairs_list.append([text_col[i], get_raw_doc_by_id(similar_col[i][j]), 1])
    return all_similar_pairs_list

In [13]:
all_similar_pairs = get_all_similar_pairs(raw_texts, similar_texts)

In [14]:
# all_similar_pairs[:2]

In [15]:
len(all_similar_pairs)

1383

#### Randomly sample equal number of negative pairs

In [16]:
# Get all negative pairs

In [17]:
# pair is negative if
# - doc2 is not in doc1's references
# - doc1 is not in doc2's references

ref_sets = [set(lst) for lst in similar_texts]

In [18]:
def get_all_negative_pairs(doc_ids, similar_sets, raw_texts):
    all_negative_pairs_list = []
    for i in range(len(doc_ids)):
        for j in range(len(doc_ids)):
            if i != j:
                doc1 = doc_ids[i]
                doc2 = doc_ids[j]
                if doc2 not in similar_sets[i]:
                    # get text
                    all_negative_pairs_list.append([remove_n_t(raw_texts[i]), remove_n_t(raw_texts[j]), 0])
    return all_negative_pairs_list

In [19]:
all_negative_pairs = get_all_negative_pairs(prep_df['doc_index'], ref_sets, raw_texts)

In [20]:
len(all_negative_pairs)

311657

In [21]:
import random

random.seed(42)
negative_pairs_samples = random.sample(all_negative_pairs, len(all_similar_pairs))

In [22]:
negative_pairs_samples[:2]

[['請蕉農注意抽穗蕉株花薊馬防治作業俾生產外觀優良品質香蕉香蕉花薊馬蟲口整年均可發現尤以蕉株抽穗期間如果疏忽花薊馬防治致使花薊馬易侵入花苞產卵危害香蕉幼果果皮造成果皮外表發生水銹斑及嚴重卵斑影響香蕉外觀品質極鉅敬請蕉農於蕉株抽穗時期能及時進行花薊馬防治以利生產高品質香蕉防治方法每35天巡視蕉園1次在花苞即將抽出莖頂時選擇下列一種葯劑配方防治(一)以2.8%第滅寧乳劑3,000倍液噴施剛抽出莖頂或初彎下花苞每隔5天噴施1次共噴3次(二)5%陶斯松或10%福瑞松粒劑(每株15公克)分別撒施於第12葉鞘內並於終花期前以2.8%第滅寧乳劑3,000倍液噴果房1次(三)終花後切除雄花苞減少蕉園花薊馬密度(四)定期防除蕉園雜草避免花薊馬滋生擴散',
  '水稻瘤野螟成蛾陸續羽化,籲請農民掌握防治時機二期稻作瘤野螟發生警報水稻瘤野螟成蛾陸續羽化台中區農業改良場提出防治對策籲請農民掌握防治時機以減少本害蟲之發生與蔓延台中縣及彰化縣之二期稻作正值孕穗初期經台中區農業改良場病蟲害發生預測發現8月中下旬瘤野螟幼蟲零星危害稻葉致使捲葉的情形9月中旬起陸續有新一代成蛾羽化本場籲請農民注意水稻瘤野螟防治工作以確保稻米生產瘤野螟又稱為稻縱捲葉蟲初孵化幼蟲啃食嫩葉葉肉形成細白線之食痕2齡後會將葉片捲成筒狀縱苞並藏匿其中取食上表皮及葉肉殘留長條白色斑痕一個捲葉內往往藏匿幼蟲1隻通常啃食數條食痕後便遷移他葉繼續危害每隻幼蟲可危害3-6片稻葉台中區農業改良場籲請農民掌握防治適期於田間成蛾發生盛期後7天進行第1次施藥或每叢稻株捲葉數平均達3葉﹝每叢稻株可檢視到1隻幼蟲﹞以上時開始施藥於第1次施藥後14天再行第2次處理可得到最好的防治效果本害蟲防治藥劑種類如下(一)2.8%賽洛寧乳劑2,000倍(二)10%得芬諾可濕性粉劑1,000倍(三)19%矽護芬水基乳劑2,000倍等其他推薦藥劑及施藥方法請參照植物保護手冊並應注意合理施用肥料避免偏施氮肥造成稻株生長過密或過於嫩綠可減少蟲害的發生與蔓延此外9月中下旬水稻易遭逢白葉枯病危害農民於發病初期可施用10%鏈四環黴素可濕性粉劑1,000倍或10%克枯爛可濕性粉劑1,000倍進行防治',
  0],
 ['氣候變化大臺南區農業改良場籲注意稻熱病發生臺南區農改場防檢局及田邊好幫手關心您近日天氣不穩定日夜溫差大部分田區旁放置之秧砧已零星發現稻熱病病斑隨著溫度日益升高

In [23]:
len(negative_pairs_samples)

1383

In [24]:
#### Combine into one dataset and shuffle
all_pairs_balanced = all_similar_pairs + negative_pairs_samples

random.shuffle(all_pairs_balanced)

In [26]:
all_pairs_balanced[:10]

[['請注意防治白葉枯病  確保水稻產量與品質本縣一期作水稻生長正值孕穗期至抽穗期間適逢高溫的環境及梅雨季節的來臨對於病蟲害的防治要特別注意目前在鹿野關山池上地區已出現白葉枯病病斑臺東區農業改良場特別呼籲農友注意防治採取必要措施以確保稻米的品質與產量白葉枯病是一種細菌性病害在高溫下繁殖極為迅速大多於分蘗盛期後至抽穗期發生病原細菌可由強風所造成的葉面傷口或水孔侵入本病初發生時沿葉緣產生黃色條斑條斑之周緣呈波浪狀或葉緣呈蒼白黃色條紋當水稻感染此病時除了葉片枯萎造成產量的損失外對稻米品質的影響甚鉅除了青米與劣米率明顯增加外白米透明度降低與蛋白質含量提高均會影響米粒外觀及食味因此在發病初期應立即防治以後每隔十天施藥一次連續三次藥劑可施用一0克枯爛可濕性粉劑一０００倍每公頃施用一．０公斤或一0鏈四環黴素可溶性粉劑一０００倍每公頃施用一．０公斤藥劑宜輪流施用以免產生抗藥性除了藥劑防治外農友們應避免在清晨露水未乾前行走於已發病之稻株間以減少人為傳播病菌低漥稻田浸水有利於細菌繁殖大雨過後應立即排水減少發病機會/fontdivid=filelink1fontsize=2face="新細明體"請點選檔名開啟或下載(依檔案大小開啟速度不同)',
  '苗栗區農業改良場發佈水稻白葉枯病警報糧食作物病蟲害發生警報中華民國90年9月21日發佈第3號一病蟲害別水稻白葉枯病二防治地點時期苗栗縣各鄉鎮市稻作栽培區水稻發病初期即開始施藥三防治應注意事項1.避免偏用氮素肥料2.晨露未乾前避免進入稻田以減少人為傳染病菌3.發病稻田於收穫後連續浸水兩週使細菌死滅四防治藥劑及方法1.藥劑名稱10%鏈四環黴素可溶性粉劑(StreptomycinTetracycline)每公頃每次施藥量1公斤稀釋倍數1000倍施藥方法發病初期開始施藥以後每隔10天施藥一次共三次2.藥劑名稱10%克枯爛可濕性粉劑(Tecloftalam)每公頃每次施藥量1.2公斤稀釋倍數1000倍施藥方法發病初期開始施藥以後每隔10天施藥一次連續三次注意事項(1)本藥劑試驗時加展著劑出來通CS-73000倍(2)收割前15天停止用藥(3)本藥劑可能發生藥害宜單劑使用不得混合6.5%鐵甲砷酸銨溶液25%賓克隆可濕性粉劑75%三賽唑可濕性粉劑或22.5%陶斯松乳劑等4種之任一種農藥或同時混合25%賓克隆可濕性粉劑及75%歐殺松乳劑二種農藥3.藥劑名稱

In [27]:
save_file_name = 'all_pairs_looser_balanced.tsv'

In [28]:
with open(save_file_name, 'w') as f:
    for lst in all_pairs_balanced:
        f.write(f'{lst[0]}\t{lst[1]}\t{lst[2]}\n')